In [ ]:
import common
from common import *

In [ ]:
traj=pa.Traj('./reference/h2o.bin')

In [ ]:
traj.getNtimesteps()

In [ ]:
traj.setAccessWindowSize(traj.getNtimesteps())
traj.setAccessStart(0)

In [ ]:
msd,gofr,sh=analyze(traj,traj,0,1000
                   )

In [ ]:
plot_gofr(0.5*0.52,3.8*0.52,gofr)

In [ ]:
plot_msd(np.arange(msd.shape[0])*50*2.4e-5,msd*0.52**2,0)

In [ ]:
density=np.array(atomic_density(traj,np.array([[1,0,0],[0,1,0],[0,0,1]])),dtype='float16')

In [ ]:
density[0].shape

In [ ]:
density_field(density[0],density[1])

In [ ]:
density_g=fft_density(density)

In [ ]:
density_g.shape

In [ ]:
density_field(density_g)